# Analysis of fitted tracks
Plots many distributions associated with fitted tracks.

## Paths
Set ``datapath`` to point to the directory which contains ``evd`` files.

In [1]:
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import h5py
import scipy.stats
import matplotlib as mpl
mpl.rc('image', cmap='viridis')

In [2]:
## path to data files
# datapath = os.path.join(os.environ['SCRATCH'],'evd/')
datapath = '/global/project/projectdirs/dune/www/data/Bern-singlecube/LArPix/dataRuns/convertedData/'
# datapath = '/global/project/projectdirs/dune/www/data/Module0-HV/LArPix/dataRuns/convertedData/test_20_12_02/'
# datapath = '/global/project/projectdirs/dune/data/larpix/processed_data/prod_20_10_22/'

## Interactive analysis

In [3]:
## helper functions
def profile_plot(x,y,xbins,method='mean',std_on=False,best_fit=False):
    binned = scipy.stats.binned_statistic(x, [y, y**2], bins=xbins, statistic=method)
    mean, mean2 = binned.statistic
    std = np.sqrt(mean2 - mean**2)
    bin_edges = binned.bin_edges
    bin_centers = (bin_edges[:-1] + bin_edges[1:])/2.

    fmt = 'r.'
    if method != 'mean': fmt = '.'
    plt.errorbar(x=bin_centers, y=mean, yerr=std if std_on else None, fmt=fmt, alpha=0.5)
    
    if best_fit:
        fit = np.polyfit(bin_centers, mean, deg=1, w=std if std_on else None)
        print('Best fit:',fit)
    
def mpv(x):
    val,bins = np.histogram(x,bins=100)
    idx = np.argmax(val)
    return (bins[idx] + bins[idx+1])/2

def vertical_assumption(d):
    rv = dict()
    theta = d['theta']
    phi = d['phi']
    
    rv['start'] = np.where(np.expand_dims(phi>0,axis=-1), d['start'], d['end'])
    rv['end'] = np.where(np.expand_dims(phi>0,axis=-1), d['end'], d['start'])
    rv['theta'] = np.where(phi>0, theta, np.pi - theta)
    rv['phi'] = np.abs(phi)
    rv['residual'] = d['residual']
    return rv

def calc_theta(xyzt):
    return np.arctan2(np.linalg.norm(xyzt[:,:2],axis=-1),xyzt[:,2])

def calc_phi(xyzt):
    return np.arctan2(xyzt[:,1],xyzt[:,0])

def azimuthal(d):
    rv = dict()
    start = d['start'][:]
    end = d['end'][:]
    start[:,[1,2]] = np.array([1,-1]) * d['start'][:,[2,1]] # swap y -> -z, z -> y
    end[:,[1,2]] = np.array([1,-1]) * d['end'][:,[2,1]] # swap y -> -z, z -> y
    theta = calc_theta(end-start)
    phi = calc_phi(end-start)
    residual = d['residual'][:]
    residual[:,[1,2]] = d['residual'][:,[2,1]] # swap y -> -z, z -> y
    
    rv['start'] = start
    rv['end'] = end
    rv['theta'] = theta
    rv['phi'] = phi
    rv['residual'] = residual
    return rv

def print_file_info(f):
    for key in f.keys():
        try:
            print(key,f[key].shape)
        except:
            print(key)
        for attr_key in f[key].attrs.keys():
            print('\t',attr_key,f[key].attrs[attr_key])


In [4]:
## look at many basic distributions from tracks
%matplotlib widget
files = sorted([os.path.basename(path) for path in glob.glob(datapath+'/*.h5')])
@widgets.interact
def display(filename=widgets.Dropdown(options=files, description="File"),
            nhit_cut=widgets.IntText(value=0, description="NHit cut"),
            length_cut=widgets.FloatText(value=0, description="Length cut"),
            residual_cut=widgets.FloatText(value=1000, description="Residual cut"),
            theta_cut=widgets.FloatText(value=0, description="Theta cut"),
            event_frac_cut=widgets.FloatText(value=0.5, description="Event fraction cut"),
            assume_vertical=widgets.ToggleButton(value=True, description="Assume vertical tracks"),
            azimuthal_coords=widgets.ToggleButton(value=False, description="Azimuthal coordinates"),
            t0_only=widgets.ToggleButton(value=True, description="T0-tagged only"),
           ):
    plt.close('all')
    f = h5py.File(os.path.join(datapath,filename),'r')
    print_file_info(f)
    if len(f['tracks']):
        t0_tagged = np.array([f['events'][ f['tracks'][i]['event_ref'] ]['n_ext_trigs'][0] 
                              for i in range(len(f['tracks']))]).astype(bool)
        event_hits = np.array([f['events'][ f['tracks'][i]['event_ref'] ]['nhit'][0] 
                               for i in range(len(f['tracks']))])
        event_fraction = f['tracks']['nhit']/event_hits
        good_track_mask = np.ones(len(f['tracks'])).astype(bool)
        good_track_mask = np.logical_and(f['tracks']['nhit'] > nhit_cut, good_track_mask)
        good_track_mask = np.logical_and(f['tracks']['length'] > length_cut, good_track_mask)
        good_track_mask = np.logical_and(np.linalg.norm(f['tracks']['residual'][:,:3],axis=-1) < residual_cut, 
                                         good_track_mask)
        good_track_mask = np.logical_and(np.abs(f['tracks']['theta']) > theta_cut, good_track_mask)
        good_track_mask = np.logical_and(np.abs(np.pi - f['tracks']['theta']) > theta_cut, good_track_mask)
        good_track_mask = np.logical_and(event_fraction > event_frac_cut, good_track_mask)
        if t0_only:
            good_track_mask = np.logical_and(t0_tagged, good_track_mask)
        
        if assume_vertical:
            v = vertical_assumption(f['tracks'])
            theta = v['theta']
            phi = v['phi']
            start = v['start']
            end = v['end']
            residual = v['residual']
        else:
            theta = f['tracks']['theta']
            phi = f['tracks']['phi']
            start = f['tracks']['start']
            end = f['tracks']['end']
            residual = f['tracks']['residual']
        if azimuthal_coords:
            a = azimuthal(f['tracks'] if not assume_vertical else v)
            theta = a['theta']
            phi = a['phi']
            start = a['start']
            end = a['end']
            residual = a['residual']
            
        plt.figure('track nhit')
        plt.hist(f['tracks']['nhit'],bins=np.linspace(0,1000,100),histtype='step')
        plt.hist(f['tracks']['nhit'][good_track_mask],bins=np.linspace(0,1000,100),histtype='step')
        plt.xlabel('nhit')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track nhit fraction')
        plt.hist(event_fraction,bins=np.linspace(0,1,100),histtype='step')
        plt.hist(event_fraction[good_track_mask],bins=np.linspace(0,1,100),histtype='step')
        plt.xlabel('fraction of event hits')
        plt.yscale('log')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track length')
        plt.hist(f['tracks']['length'],bins=np.linspace(0,520,100),histtype='step')
        plt.hist(f['tracks']['length'][good_track_mask],bins=np.linspace(0,520,100),histtype='step')
        plt.xlabel('track length [mm]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track theta')
        plt.hist(theta,bins=np.linspace(0,np.pi,100),histtype='step')
        plt.hist(theta[good_track_mask],bins=np.linspace(0,np.pi,100),histtype='step')
        plt.xlabel('track theta [rad.]')
        plt.legend(('all tracks','selected tracks'))
        plt.axvline(np.pi/2)
        
        plt.figure('track phi')
        plt.hist(phi,bins=np.linspace(-np.pi,np.pi,100),histtype='step')
        plt.hist(phi[good_track_mask],bins=np.linspace(-np.pi,np.pi,100),histtype='step')
        plt.xlabel('track phi [rad.]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track res x')
        plt.hist(residual[:,0],bins=np.linspace(0,10,100),histtype='step')
        plt.hist(residual[good_track_mask,0],bins=np.linspace(0,10,100),histtype='step')
        plt.xlabel('track residual x [mm]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track res y')
        plt.hist(residual[:,1],bins=np.linspace(0,10,100),histtype='step')
        plt.hist(residual[good_track_mask,1],bins=np.linspace(0,10,100),histtype='step')
        plt.xlabel('track residual y [mm]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track res z')
        plt.hist(residual[:,2],bins=np.linspace(0,10,100),histtype='step')
        plt.hist(residual[good_track_mask,2],bins=np.linspace(0,10,100),histtype='step')
        plt.xlabel('track residual z [mm]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track res')
        plt.hist(np.linalg.norm(residual[:,:3],axis=-1),bins=np.linspace(0,10,100),histtype='step')
        plt.hist(np.linalg.norm(residual[good_track_mask,:3],axis=-1),bins=np.linspace(0,10,100),
                 histtype='step')
        plt.xlabel('track residual [mm]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track q')
        plt.hist(f['tracks']['q']*0.250,bins=np.linspace(0,1e3,100),histtype='step')
        plt.hist(f['tracks']['q'][good_track_mask]*0.250,bins=np.linspace(0,1e3,100),
                 histtype='step')
        plt.xlabel('track q [ke]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track time length')
        plt.hist(f['tracks']['ts_end'] - f['tracks']['ts_start'],bins=np.linspace(0,2000,100),
                 histtype='step')
        plt.hist((f['tracks']['ts_end'] - f['tracks']['ts_start'])[good_track_mask],
                 bins=np.linspace(0,2000,100),histtype='step')
        plt.xlabel('track dt [0.1us]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track depth')
        plt.hist(np.abs(end[:,2] - start[:,2]),bins=np.linspace(0,520,100),histtype='step')
        plt.hist(np.abs(end[:,2] - start[:,2])[good_track_mask],bins=np.linspace(0,520,100),histtype='step')
        plt.xlabel('track depth [mm]')
        plt.legend(('all tracks','selected tracks'))
        
        plt.figure('track dQ/dx')
        dqdx = f['tracks']['q']*0.250/f['tracks']['length']
        dqdx[~np.isfinite(dqdx)] = -1
        plt.hist(dqdx,bins=np.linspace(0,20,100),histtype='step')
        plt.hist(dqdx[good_track_mask],bins=np.linspace(0,20,100),histtype='step')
        plt.xlabel('track mean dQ/dx [ke/mm]')
        plt.legend(('all tracks','selected tracks'))

#         good_track_mask = np.ones(len(f['tracks'])).astype(bool)
        
        x_start = start[good_track_mask,0]
        y_start = start[good_track_mask,1]
        z_start = start[good_track_mask,2]
        x_end = end[good_track_mask,0]
        y_end = end[good_track_mask,1]
        z_end = end[good_track_mask,2]
        plt.figure('track end points (x,y)')
        fig, axes = plt.subplots(1, 2, num='track end points (x,y)', sharex=True, sharey=True)
        axes[0].hist2d(x_start,y_start,
                       bins=(np.linspace(-166.275,166.275,75),np.linspace(-166.275,166.275,75)),cmin=0.5)
        plt.colorbar(axes[1].hist2d(x_end,y_end,
                                    bins=(np.linspace(-166.275,166.275,75),np.linspace(-166.275,166.275,75)),
                                    cmin=0.5)[-1])
        plt.xlabel('x [mm]')
        axes[0].set_ylabel('y [mm]')
        axes[0].set_title('track start')
        axes[1].set_title('track end')
        fig, axes = plt.subplots(1, 2, num='track end points (x,z)', sharex=True, sharey=True)
        axes[0].hist2d(x_start,z_start,
                       bins=(np.linspace(-166.275,166.275,75),
                             np.linspace(-166.275*0,166.275*2,75)),
                       cmin=0.5)
        plt.colorbar(axes[1].hist2d(x_end,z_end,
                                    bins=(np.linspace(-166.275,166.275,75),
                                          np.linspace(-166.275*0,166.275*2,75)),
                                    cmin=0.5)[-1])
        plt.xlabel('x [mm]')
        axes[0].set_ylabel('z [mm]')
        axes[0].set_title('track start')
        axes[1].set_title('track end')
        fig, axes = plt.subplots(1, 2, num='track end points (y,z)', sharex=True, sharey=True)
        axes[0].hist2d(z_start,y_start,
                       bins=(np.linspace(-166.275*0,166.275*2,75),np.linspace(-166.275,166.275,75)),cmin=0.5)
        plt.colorbar(axes[1].hist2d(z_end,y_end,
                                    bins=(np.linspace(-166.275*0,166.275*2,75),
                                          np.linspace(-166.275,166.275,75)),
                                    cmin=0.5)[-1])
        plt.xlabel('z [mm]')
        axes[0].set_ylabel('y [mm]')
        axes[0].set_title('track start')
        axes[1].set_title('track end')
        
        plt.figure('track dQ/dx v. theta')
        plt.hist2d(theta[good_track_mask],dqdx[good_track_mask],
                   bins=(np.linspace(0,np.pi,100),np.linspace(0,20,100)))
        plt.colorbar()
        profile_plot(theta[good_track_mask],dqdx[good_track_mask],np.linspace(0,np.pi,100))
        profile_plot(theta[good_track_mask],dqdx[good_track_mask],np.linspace(0,np.pi,100), 
                     method='median')
        profile_plot(theta[good_track_mask],dqdx[good_track_mask],np.linspace(0,np.pi,100), 
                     method=mpv)
        plt.legend(['mean','median','mpv'])
        plt.ylabel('track mean dQ/dx [ke/mm]')
        plt.xlabel('track theta [rad.]')
        plt.axvline(np.pi/2)
        
        plt.figure('track dQ/dx v. phi')
        plt.hist2d(phi[good_track_mask],dqdx[good_track_mask],
                   bins=(np.linspace(-np.pi,np.pi,100),np.linspace(0,20,100)))
        plt.colorbar()
        profile_plot(phi[good_track_mask],dqdx[good_track_mask],np.linspace(-np.pi,np.pi,100))
        profile_plot(phi[good_track_mask],dqdx[good_track_mask],np.linspace(-np.pi,np.pi,100), 
                     method='median')
        profile_plot(phi[good_track_mask],dqdx[good_track_mask],np.linspace(-np.pi,np.pi,100), 
                     method=mpv)
        plt.legend(['mean','median','mpv'])
        plt.ylabel('track mean dQ/dx [ke/mm]')
        plt.xlabel('track phi [rad.]')
        
        plt.figure('track dQ/dx v. residual')
        plt.hist2d(np.linalg.norm(residual[good_track_mask,:3],axis=-1),dqdx[good_track_mask],
                bins=(np.linspace(0,10,100),np.linspace(0,20,100)))
        plt.colorbar()
        profile_plot(np.linalg.norm(residual[good_track_mask,:3],axis=-1),dqdx[good_track_mask],
                     np.linspace(0,10,100), method='mean')
        profile_plot(np.linalg.norm(residual[good_track_mask,:3],axis=-1),dqdx[good_track_mask],
                     np.linspace(0,10,100), method='median')
        profile_plot(np.linalg.norm(residual[good_track_mask,:3],axis=-1),dqdx[good_track_mask],
                     np.linspace(0,10,100), method=mpv)
        plt.legend(['mean','median','mpv'])
        plt.ylabel('track mean dQ/dx [ke/mm]')
        plt.xlabel('track residual [mm]')
        
        plt.figure('track dQ/dx v. length')
        plt.hist2d(f['tracks']['length'][good_track_mask],dqdx[good_track_mask],
                   bins=(np.linspace(0,520,100),np.linspace(0,20,100)))
        plt.colorbar()
        profile_plot(f['tracks']['length'][good_track_mask],dqdx[good_track_mask],np.linspace(0,520,100), 
                     method='mean')
        profile_plot(f['tracks']['length'][good_track_mask],dqdx[good_track_mask],np.linspace(0,520,100), 
                     method='median')
        profile_plot(f['tracks']['length'][good_track_mask],dqdx[good_track_mask],np.linspace(0,520,100), 
                     method=mpv)
        plt.legend(['mean','median','mpv'])
        plt.ylabel('track mean dQ/dx [ke/mm]')
        plt.xlabel('track length [mm]')
        
        plt.figure('track dQ/dx v. Q')
        plt.hist2d(f['tracks']['q'][good_track_mask],dqdx[good_track_mask],
                   bins=(np.linspace(0,1e3,100),np.linspace(0,20,100)))
        plt.colorbar()
        profile_plot(f['tracks']['q'][good_track_mask],dqdx[good_track_mask],np.linspace(0,1e3,100), 
                     method='mean')
        profile_plot(f['tracks']['q'][good_track_mask],dqdx[good_track_mask],np.linspace(0,1e3,100), 
                     method='median')
        profile_plot(f['tracks']['q'][good_track_mask],dqdx[good_track_mask],np.linspace(0,1e3,100), 
                     method=mpv)
        plt.legend(['mean','median','mpv'])
        plt.ylabel('track mean dQ/dx [ke/mm]')
        plt.xlabel('track Q [ke]')
        
        plt.figure('track dQ/dx v. Z')
        z = (f['tracks']['start'][:,2] + f['tracks']['end'][:,2])/2
        plt.hist2d(z[good_track_mask],dqdx[good_track_mask],
                   bins=(np.linspace(0,500,100),np.linspace(0,20,100)))
        plt.colorbar()
        profile_plot(z[good_track_mask],dqdx[good_track_mask],np.linspace(0,500,100), method='mean')
        profile_plot(z[good_track_mask],dqdx[good_track_mask],np.linspace(0,500,100), method='median')
        profile_plot(z[good_track_mask],dqdx[good_track_mask],np.linspace(0,500,100), method=mpv)
        plt.legend(['mean','median','mpv'])
        plt.ylabel('track mean dQ/dx [ke/mm]')
        plt.xlabel('track <z> [mm]')
        
        plt.figure('track residual v. theta')
        plt.hist2d(theta[good_track_mask],np.linalg.norm(residual[good_track_mask,:3],axis=-1),
                   bins=(np.linspace(0,np.pi,100),np.linspace(0,10,100)))
        plt.colorbar()
        profile_plot(theta[good_track_mask],np.linalg.norm(residual[good_track_mask,:3],axis=-1),
                     np.linspace(0,np.pi,100))
        plt.ylabel('track residual [mm]')
        plt.xlabel('track theta [rad.]')
        plt.axvline(np.pi/2)
        
        plt.figure('track length v. theta')
        plt.hist2d(theta[good_track_mask],f['tracks']['length'][good_track_mask],
                   bins=(np.linspace(0,np.pi,100),np.linspace(0,440,100)))
        plt.colorbar()
        profile_plot(theta[good_track_mask],f['tracks']['length'][good_track_mask],np.linspace(0,np.pi,100))
        plt.ylabel('track length [mm]')
        plt.xlabel('track theta [rad.]')
        plt.axvline(np.pi/2)
        
        plt.figure('track length v. residual')
        plt.hist2d(f['tracks']['length'][good_track_mask],
                   np.linalg.norm(residual[good_track_mask,:3],axis=-1),
                   bins=(np.linspace(0,520,100),np.linspace(0,10,100)))
        plt.colorbar()
        profile_plot(f['tracks']['length'][good_track_mask],
                     np.linalg.norm(residual[good_track_mask,:3],axis=-1),
                     np.linspace(0,520,100))
        plt.xlabel('track length [mm]')
        plt.ylabel('track residual [mm]')
        
        plt.figure('track charge v. nhit')
        plt.hist2d(f['tracks']['nhit'][good_track_mask],f['tracks']['q'][good_track_mask]*0.250,
                   bins=(np.linspace(0,100,100),np.linspace(0,1e3,100)))
        plt.colorbar()
        profile_plot(f['tracks']['nhit'][good_track_mask],f['tracks']['q'][good_track_mask]*0.250,
                     np.linspace(0,100,100))
        plt.xlabel('track n hits')
        plt.ylabel('track q [ke]')
        
        plt.figure('track dQ/dx v. mean hit q')
        q_mean = f['tracks']['q']*0.250/f['tracks']['nhit']
        plt.hist2d(q_mean[good_track_mask],dqdx[good_track_mask],
                   bins=(np.linspace(0,30,100),np.linspace(0,20,100)))
        plt.colorbar()
        profile_plot(q_mean[good_track_mask],dqdx[good_track_mask],np.linspace(0,30,100))
        profile_plot(q_mean[good_track_mask],dqdx[good_track_mask],np.linspace(0,30,100), method='median')
        profile_plot(q_mean[good_track_mask],dqdx[good_track_mask],np.linspace(0,30,100), method=mpv)
        plt.legend(['mean','median','mpv'])
        plt.xlabel('track mean hit q [ke]')
        plt.ylabel('track dQ/dx [ke/mm]')
        
        plt.figure('track dQ/dx v. nhit')
        plt.hist2d(f['tracks']['nhit'][good_track_mask],dqdx[good_track_mask],
                   bins=(np.linspace(0,100,100),np.linspace(0,20,100)))
        plt.colorbar()
        profile_plot(f['tracks']['nhit'][good_track_mask],dqdx[good_track_mask],np.linspace(0,100,100))
        profile_plot(f['tracks']['nhit'][good_track_mask],dqdx[good_track_mask],np.linspace(0,100,100),
                     method='median')
        profile_plot(f['tracks']['nhit'][good_track_mask],dqdx[good_track_mask],np.linspace(0,100,100),
                     method=mpv, best_fit=True)
        plt.legend(['mean','median','mpv'])
        plt.xlabel('track n hits')
        plt.ylabel('track dQ/dx [ke/mm]')
        
        plt.figure('track dQ/dx v. event fraction')
        plt.hist2d(event_fraction[good_track_mask],dqdx[good_track_mask],
                   bins=(np.linspace(0,1,100),np.linspace(0,20,100)))
        plt.colorbar()
        profile_plot(event_fraction[good_track_mask],dqdx[good_track_mask],np.linspace(0,1,100))
        profile_plot(event_fraction[good_track_mask],dqdx[good_track_mask],np.linspace(0,1,100),
                     method='median')
        profile_plot(event_fraction[good_track_mask],dqdx[good_track_mask],np.linspace(0,1,100),
                     method=mpv, best_fit=True)
        plt.legend(['mean','median','mpv'])
        plt.xlabel('event fraction')
        plt.ylabel('track dQ/dx [ke/mm]')
    else:
        print('no events in file')

interactive(children=(Dropdown(description='File', options=('datalog_2020_10_19_10_13_39_CEST_evd.h5', 'datalo…

## External trigger analysis

In [11]:
## look at external trigger counts and correlations
%matplotlib widget
files = sorted([os.path.basename(path) for path in glob.glob(datapath+'/*.h5')])
@widgets.interact
def display(filename=widgets.Dropdown(options=files, description="File")
           ):
    plt.close('all')
    f = h5py.File(os.path.join(datapath,filename),'r')
    print_file_info(f)
    events=f['events']
    hits=f['hits']
    ext_trigs=f['ext_trigs']
    
    mask = events['n_ext_trigs'] > 0
    dt = np.empty((0,))
    dt_all = np.empty((0,))
    for ev in events[mask]:
        if ev['n_ext_trigs'] > 0:
            trig_ts = ext_trigs[ ev['ext_trig_ref'] ]['ts']
            for ts in trig_ts:
                dt = np.concatenate((hits[ ev['hit_ref'] ]['ts'] - ts, dt), axis=0)
    for trig in ext_trigs:
        ts = trig['ts']
        delta = hits['ts']-ts
        dt_all = np.concatenate((delta[np.abs(delta)<2e3*5], dt_all), axis=0)
            
    plt.figure('relative time (all hits in event)')
    plt.hist(dt,bins=np.linspace(-2e3*5,2e3*5,100),histtype='step')
    plt.xlabel('relative timestamp [0.1us]')
    plt.ylabel('count')
    plt.title(filename)
    
    plt.axvline(0,color='k')
    plt.axvline(1820,color='k')
    
    plt.figure('relative time (all hits in file)')
    plt.hist(dt_all,bins=np.linspace(-2e3*5,2e3*5,100),histtype='step')
    plt.xlabel('relative timestamp [0.1us]')
    plt.ylabel('count')
    plt.title(filename)
    
    plt.axvline(0,color='k')
    plt.axvline(1820,color='k')


interactive(children=(Dropdown(description='File', options=('datalog_2020_11_25_06_04_19_CET_evd.h5', 'datalog…

## Run stability analysis

In [ ]:
## data cache so you don't have to reload files every time you update plots
cache = {}

In [ ]:
## the run files to plot over time
files = sorted(glob.glob(os.path.join(datapath,'*.h5')))

In [ ]:
# files = [
#     'datalog_2020_10_28_22_41_18_CET_evd.h5',
#     'datalog_2020_10_28_23_11_19_CET_evd.h5',
#     'datalog_2020_10_28_23_41_19_CET_evd.h5',
#     'datalog_2020_10_29_00_11_19_CET_evd.h5',
#     'datalog_2020_10_29_01_02_44_CET_evd.h5',
#     'datalog_2020_10_29_02_54_52_CET_evd.h5',
#     'datalog_2020_10_29_03_24_52_CET_evd.h5',
#     'datalog_2020_10_29_03_54_53_CET_evd.h5',
#     'datalog_2020_10_29_04_24_53_CET_evd.h5',
#     'datalog_2020_10_29_05_05_01_CET_evd.h5',
#     'datalog_2020_10_29_05_35_03_CET_evd.h5',
#     'datalog_2020_10_29_06_06_40_CET_evd.h5',
#     'datalog_2020_10_29_06_36_42_CET_evd.h5',
#     'datalog_2020_10_29_07_17_11_CET_evd.h5',
#     'datalog_2020_10_29_07_47_11_CET_evd.h5',
#     'datalog_2020_10_29_08_17_11_CET_evd.h5',
#     'datalog_2020_10_29_08_47_11_CET_evd.h5',
#     'datalog_2020_10_29_09_08_24_CET_evd.h5',
#     'datalog_2020_10_29_09_38_24_CET_evd.h5',
#     'datalog_2020_10_29_10_08_24_CET_evd.h5',
#     'datalog_2020_10_29_10_38_24_CET_evd.h5',
#     'datalog_2020_10_30_10_05_32_CET_evd.h5',
#     'datalog_2020_10_30_10_35_32_CET_evd.h5',
#     'datalog_2020_10_30_11_13_05_CET_evd.h5',
#     'datalog_2020_10_30_13_04_04_CET_evd.h5',
#     'datalog_2020_10_30_13_34_05_CET_evd.h5',
#     'datalog_2020_10_30_14_04_05_CET_evd.h5',
#     'datalog_2020_10_30_14_34_06_CET_evd.h5',
#     'datalog_2020_10_30_15_32_26_CET_evd.h5',
#     'datalog_2020_10_30_16_02_31_CET_evd.h5',
#     'datalog_2020_10_30_16_35_21_CET_evd.h5',
#     'datalog_2020_10_30_17_06_22_CET_evd.h5',
#     'datalog_2020_10_30_17_56_15_CET_evd.h5',
#     'datalog_2020_10_30_18_26_16_CET_evd.h5',
#     'datalog_2020_10_30_18_56_16_CET_evd.h5',
#     'datalog_2020_10_30_19_26_17_CET_evd.h5',
#     'datalog_2020_10_30_20_15_12_CET_evd.h5',
#     'datalog_2020_10_30_20_48_18_CET_evd.h5',
#     'datalog_2020_10_30_21_21_19_CET_evd.h5',
#     'datalog_2020_10_30_21_54_04_CET_evd.h5',
#     'datalog_2020_10_30_22_28_00_CET_evd.h5',
#     'datalog_2020_10_30_23_01_07_CET_evd.h5',
#     'datalog_2020_10_30_23_37_11_CET_evd.h5'
# ]

# lifetimes = [
#     (1.03,	0.17,	0.13,	'datalog_2020_10_28_22_41_18_CET_evd.h5'),
#     (1.03,	0.17,	0.13,	'datalog_2020_10_28_23_11_19_CET_evd.h5'),
#     (1.04,	0.18,	0.13,	'datalog_2020_10_28_23_41_19_CET_evd.h5'),
#     (1.25,	0.26,	0.18,	'datalog_2020_10_29_00_11_19_CET_evd.h5'),
#     (1.5,   0.4,	0.26,	'datalog_2020_10_29_01_02_44_CET_evd.h5'),
#     (0.8,   0.1,	0.08,	'datalog_2020_10_29_02_54_52_CET_evd.h5'),
#     (1.15,	0.23,	0.17,	'datalog_2020_10_29_03_24_52_CET_evd.h5'),
#     (1.28,	0.29,	0.2,	'datalog_2020_10_29_03_54_53_CET_evd.h5'),
#     (0.91,	0.13,	0.1,	'datalog_2020_10_29_04_24_53_CET_evd.h5'),
#     (0.84,	0.12,	0.09,	'datalog_2020_10_29_07_17_11_CET_evd.h5'),
#     (1.03,	0.17,	0.13,	'datalog_2020_10_29_07_47_11_CET_evd.h5'),
#     (0.83,	0.11,	0.09,	'datalog_2020_10_29_08_17_11_CET_evd.h5'),
#     (0.8,   0.11,	0.08,	'datalog_2020_10_29_09_08_24_CET_evd.h5'),
#     (0.77,	0.1,    0.08,	'datalog_2020_10_29_09_38_24_CET_evd.h5'),
#     (0.87,	0.12,	0.09,	'datalog_2020_10_29_10_08_24_CET_evd.h5'),
#     (0.81,	0.11,	0.08,	'datalog_2020_10_29_10_38_24_CET_evd.h5'),
#     (0.42,	0.03,	0.03,	'datalog_2020_10_30_10_05_32_CET_evd.h5'),
#     (0.48,	0.04,	0.03,	'datalog_2020_10_30_10_35_32_CET_evd.h5'),
#     (0.54,	0.08,	0.06,	'datalog_2020_10_30_11_13_05_CET_evd.h5'),
#     (0.76,	0.1,	0.08,	'datalog_2020_10_30_13_04_04_CET_evd.h5'),
#     (0.79,	0.11,	0.09,	'datalog_2020_10_30_13_34_05_CET_evd.h5'),
#     (0.86,	0.13,	0.1,	'datalog_2020_10_30_14_04_05_CET_evd.h5'),
#     (0.86,	0.13,	0.1,	'datalog_2020_10_30_14_34_06_CET_evd.h5'),
#     (1.26,	0.27,	0.19,	'datalog_2020_10_30_17_56_15_CET_evd.h5'),
#     (1.36,	0.34,	0.23,	'datalog_2020_10_30_18_26_16_CET_evd.h5'),
#     (1.14,	0.22,	0.16,	'datalog_2020_10_30_18_56_16_CET_evd.h5'),
#     (1.05,	0.17,	0.13,	'datalog_2020_10_30_19_26_17_CET_evd.h5'),
#     (1.25,	0.27,	0.19,	'datalog_2020_10_30_20_15_12_CET_evd.h5'),
#     (1.14,	0.21,	0.15,	'datalog_2020_10_30_20_48_18_CET_evd.h5'),
#     (1.16,	0.22,	0.16,	'datalog_2020_10_30_21_21_19_CET_evd.h5'),
#     (1.59,	0.46,	0.29,	'datalog_2020_10_30_21_54_04_CET_evd.h5'),
#     (1.38,	0.34,	0.23,	'datalog_2020_10_30_22_28_00_CET_evd.h5'),
#     (1.33,	0.3,    0.21,	'datalog_2020_10_30_23_01_07_CET_evd.h5'),
#     (1.07,	0.19,	0.14,	'datalog_2020_10_30_23_37_11_CET_evd.h5')
# ]
# lifetimes = dict([(lt[-1],lt[0]) for lt in lifetimes])
print(len(files)*30,'min of good data')

In [ ]:
%matplotlib widget
## Ionization calculations
K = 0.307075*100. # MeV mm^2 / mol

Z = 18
A = 39.948 # [https://pdg.lbl.gov/2020/AtomicNuclearProperties/HTML/liquid_argon.html]
I = 188.0e-6 # MeV [https://physics.nist.gov/PhysRefData/XrayMassCoef/tab1.html]
rho = 1.396/1000. # g/mm^3
w = 23.6e-6 # MeV/ionization
r = 0.6980 # recombination factor [https://lar.bnl.gov/properties/]

m_e = 0.510998 # MeV/c^2
m_muon = 105.7 # MeV/c^2

def landau_csi(x,p,m=m_muon):
    ''' x is ionization length [cm], p is particle momentum [MeV/c], m is particle mass [MeV/c^2] '''
    beta = p / np.sqrt(p**2 + m**2)
    return (K/2) * (Z/A) * (x*rho/beta**2)

def delta(betagamma):
    ''' for values: [https://pdg.lbl.gov/2020/AtomicNuclearProperties/MUE/muE_liquid_argon.pdf] '''
#     x0 = 1.7635 # https://lar.bnl.gov/properties/pass.html ref [1]
#     x1 = 4.485
#     Cm = 11.9480
#     a  = 0.19714
#     k  = 2.9618
    x0 = 0.2000  # [pdg]
    x1 = 3.000   # [pdg]
    Cm = 5.2146  # [pdg]
    a  = 0.19559 # [pdg]
    k  = 3.000   # [pdg]
    x = np.log10(betagamma)
    if x < x0:
        return 0.
    elif x >= x0 and x < x1:
        return 2*np.log(10)*x - Cm + a*(x1-x)**k
    return 2*np.log(10)*x - Cm

def landau_mpv(x,p,m=m_muon):
    e = np.sqrt(p**2 + m**2)
    beta = p / e
    gamma = e / m
    csi = landau_csi(x,p,m)
    c0 = np.log(2 * m_e * beta**2 * gamma**2 / I)
    c1 = np.log(csi/I)
    return  csi * (c0 + c1 + 0.200 - beta**2 - delta(beta*gamma))

x = 310 # mm
p = 4e3 # MeV/c
std_2_fwhm = 2*np.sqrt(2*np.log(2)) # convert btw FWHM and sigma for a gaussian

print('\n~~~ INFO ~~~')
print('using track length of {}mm'.format(x))
print('using muon momentum of {}MeV/c'.format(p))

print('\n~~~ PEAK ~~~')
print('expected {:0.03f}MeV/mm [mpv]'.format(landau_mpv(x,p)/x))
expected_peak = (landau_mpv(x,p) / w) * r / x / 1e3 # ke/mm
print('expected {:0.03f}ke/mm [mpv]'.format(expected_peak))
meas_peak = 6.29 # ke/mm
print('meas {:0.03f}ke/mm [mpv]'.format(meas_peak))
diff_peak = meas_peak/expected_peak - 1.
print('scale off by {:0.03}%'.format(diff_peak*100.))

print('\n~~~ WIDTH ~~~')
meas_sigma = 0.4685 # ke/mm
print('meas {:0.03f}ke/mm [std]'.format(meas_sigma))
landau_fwhm = (4 * landau_csi(x,p) / w) * r / x # e/cm
landau_sigma = landau_fwhm / 1e3 / std_2_fwhm # ke/mm
print('expected energy fluctuations {:0.03f}ke/mm [std]'.format(landau_sigma))
pixel_pitch = 4.434 # mm
noise = 1. # ke
noise_sigma = (noise) * np.sqrt(x/pixel_pitch) / x
print('expected noise {:0.03f}ke/mm [std]'.format(noise_sigma))
fano_sigma = np.sqrt(.107) * np.sqrt(expected_peak*x) / x
print('expected stat {:0.03f}ke/mm [std]'.format(fano_sigma))
expected_sigma = np.sqrt(landau_sigma**2 + noise_sigma**2 + fano_sigma**2)
diff_sigma = np.sqrt(np.abs(expected_sigma**2 - meas_sigma**2))
print('total expected {:0.03f}ke/mm [std]'.format(expected_sigma))
print('remaining {:0.03f}ke/mm [std]'.format(diff_sigma))
print('variance off by {:0.03f}%'.format((diff_sigma**2/expected_sigma**2)*100.))


def mpv(x,p):
    return (landau_mpv(x,p) / w) * r / x / 1e3
plt.close('MVP length dep.')
x = np.linspace(100,500,100)
plt.figure('MVP length dep.')
p = 1e3
plt.plot(x,mpv(x,p),label='1GeV/c muon')
p = 2e3
plt.plot(x,mpv(x,p),label='2GeV/c muon')
p = 4e3
plt.plot(x,mpv(x,p),label='4GeV/c muon')
p = 10e3
plt.plot(x,mpv(x,p),label='10GeV/c muon')
plt.xlabel('track length [mm]')
plt.ylabel('dQ/dx MPV [ke/mm]')
plt.grid(1)
plt.legend()
plt.tight_layout()

def width(x,p,verbose=False):
    fwhm_edep = (4 * landau_csi(x,p) / w) * r / x
    sigma_edep = fwhm_edep / 1e3 / std_2_fwhm
    if verbose: print('edep:',sigma_edep)
    expected_peak = (landau_mpv(x,p) / w) * r / x / 1e3
    sigma_fano = np.sqrt(.107) * np.sqrt(expected_peak*x) / x
    if verbose: print('stat:',sigma_fano)
    sigma_noise = (noise/x) * np.sqrt(x/pixel_pitch)
    if verbose: print('noise:',sigma_noise)
    total = np.sqrt(sigma_edep**2 + sigma_fano**2 + sigma_noise**2)
    if verbose: print('total:',total)
    return total
plt.close('sigma length dep.')
x = np.linspace(100,500,100)
plt.figure('sigma length dep.')
p = 1e3
plt.plot(x,width(x,p),label='1GeV/c muon')
p = 2e3
plt.plot(x,width(x,p),label='2GeV/c muon')
p = 4e3
plt.plot(x,width(x,p),label='4GeV/c muon')
p = 10e3
plt.plot(x,width(x,p),label='10GeV/c muon')
plt.xlabel('track length [mm]')
plt.ylabel('dQ/dx sigma [ke/mm]')
plt.grid(1)
plt.legend()
plt.tight_layout()

In [ ]:
## generate plots of dqdx mean and width over the full run
%matplotlib widget
import time
import matplotlib.dates as md
from scipy.interpolate import UnivariateSpline
import scipy.optimize as optimize
import datetime as dt

# source_file_path = '/global/project/projectdirs/dune/www/data/Bern-singlecube/LArPix/dataRuns/rawData/'
# def fetch_runtimes(f):
#     source_file = f['info'].attrs['source_file']
#     with h5py.File(os.path.join(source_file_path,source_file),'r') as raw_f:
#         return raw_f['_header'].attrs['created'], raw_f['_header'].attrs['modified']
    
def fwhm(vals, max_bin, bins):
    spline = UnivariateSpline((bins[:-1]+bins[1:])/2, vals-vals[max_bin]*0.5, s=0)
    r = np.array(spline.roots())
    if len(r) < 2: return 0.
    d = np.abs(r - bins[max_bin])
    lower = r - bins[max_bin] < 0
    upper = ~lower
    r_lower = r[lower][np.argmin(d[lower])]
    r_upper = r[upper][np.argmin(d[upper])]
    return np.abs(r_lower - r_upper)

def gaus(x,peak,mean,sigma,*args):
    return peak * np.exp(-0.5 * (x-mean)**2 / (sigma**2))

@widgets.interact
def display(filenames=widgets.SelectMultiple(options=list(reversed(sorted(files))), rows=10, 
                                             description="File(s)"),
            nbins=widgets.IntText(value=100, description="N bins"),
            dqdx_max=widgets.FloatText(value=20, description="dQ/dx max bin [ke/mm]"),
           ):
    plt.close('all')
    if len(filenames) == 0: return

    for filename in filenames:
        if not filename in cache and os.path.exists(os.path.join(datapath,filename)):
            with h5py.File(os.path.join(datapath,filename),'r') as f:
                cache[filename] = dict()
                if (len(f['tracks'])
                        and 'ext_trigs' in f.keys() 
#                         and filename in lifetimes 
                        and 'electron_lifetime_file' in f['info'].attrs):
                    print('\nloading from file ({})...'.format(filename))
                    start_time = time.time()
                    
                    tracks = f['tracks']
                    hits = f['hits']
                    events = f['events']
                    
#                     s,e = fetch_runtimes(f)
                    s,e = np.min(events['unix_ts']),np.max(events['unix_ts'])
                    cache[filename]['run_start'] = s
                    cache[filename]['run_end'] = e
                    print('start:',s,'\nend:',e)
                    
                    good_track_mask = np.ones(len(tracks)).astype(bool)
#                     good_track_mask = np.logical_and(tracks['length'] > 100, good_track_mask)
                    good_track_mask = np.logical_and(tracks['length'] > 300, good_track_mask)
                    good_track_mask = np.logical_and(tracks['length'] < 320, good_track_mask)
                    good_track_mask = np.logical_and(tracks['nhit'] > 7, good_track_mask)
                    print('Initial tracks:',np.sum(good_track_mask),
                          ' ({:0.02f}s)'.format(time.time()-start_time))
                    
                    event_ref = tracks[good_track_mask]['event_ref']
                    event_hits = np.array([events[ref]['nhit'][0] 
                                           for ref in event_ref])
                    event_fraction = tracks['nhit'][good_track_mask]/event_hits
                    good_track_mask[good_track_mask] = event_fraction > 0.5
                    event_ref = tracks[good_track_mask]['event_ref']
                    print('Good tracks:',np.sum(good_track_mask),
                          ' ({:0.02f}s)'.format(time.time()-start_time))
                    
                    ext_tagged = np.array([events[ref]['n_ext_trigs'][0] 
                                          for ref in event_ref]).astype(bool)
                    t0_tagged = np.logical_or(ext_tagged, np.logical_and(
                            tracks['end'][good_track_mask,-1] - tracks['start'][good_track_mask,-1] < 1840,
                            tracks['end'][good_track_mask,-1] - tracks['start'][good_track_mask,-1] > 1800
                    ))
                    t0_tagged = ext_tagged
                    good_t0_track_mask = good_track_mask.copy()
                    good_t0_track_mask[good_track_mask] = t0_tagged
                    print('t0-tagged tracks:',np.sum(good_t0_track_mask),' ({:0.02f}s)'.format(time.time()\
                                                                                               -start_time))

                    dqdx = (tracks['q'][good_t0_track_mask] * 0.250
                        / tracks['length'][good_t0_track_mask])
                    dqdx = dqdx[np.isfinite(dqdx)]
                    vals,bins = np.histogram(dqdx,bins=np.linspace(0,dqdx_max,nbins))
                    idx = min(np.argmax(vals),len(bins)-2)
                    bin_centers = (bins[:-1] + bins[1:])/2
                    fit_range = slice(int(np.clip(idx-.05*nbins,0,len(bin_centers)-1)),
                                      int(np.clip(1+idx+.05*nbins,0,len(bin_centers)-1)))
                    params,cov = optimize.curve_fit(gaus, bin_centers[fit_range], vals[fit_range], 
                                            p0=[vals[idx],6.,1.], 
                                            sigma=np.clip(np.sqrt(vals[fit_range]),1,None))
                    print(filename, 'fit params:', params, 'fit covariance:', cov)
                    print('fit: {}\tcov: {}'.format(params,cov),
                          ' ({:0.02f}s)'.format(time.time()-start_time))
                    
                    cache[filename]['track_length_dist'] = np.histogram(tracks['length'][good_t0_track_mask],bins=np.linspace(0,550,100))
                    cache[filename]['dqdx_dist'] = (vals,bins)
                    cache[filename]['dqdx_mean'] = np.mean(dqdx)
                    cache[filename]['dqdx_std'] = np.std(dqdx)
                    cache[filename]['dqdx_mvp'] = (bins[idx] + bins[idx+1])/2
                    cache[filename]['dqdx_fwhm'] = fwhm(vals,idx,bins)
                    cache[filename]['dqdx_fit_range'] = (bin_centers[fit_range][0], 
                                                         bin_centers[fit_range][-1])
                    cache[filename]['dqdx_fit_peak'] = params[0]
                    cache[filename]['dqdx_fit_mean'] = params[1]
                    cache[filename]['dqdx_fit_width'] = params[2]
                    cache[filename]['dqdx_fit_cov'] = cov
                    cache[filename]['light_eff'] = np.sum(ext_tagged) \
                        / (np.sum(good_track_mask)+1e-9)
                    cache[filename]['light_trigger_rate'] = len(f['ext_trigs'])/(e-s+1e-9)
                    cache[filename]['good_track_rate'] = np.sum(good_track_mask)/(e-s+1e-9)
        if filename in cache:
#             print('loading from cache ({})...'.format(filename))
            if not all([key in cache[filename] for key in('dqdx_mean','dqdx_std','dqdx_mvp','dqdx_fwhm',
                                                          'light_eff','run_start','run_end')]):
                del cache[filename]
            
    print('mean dq/dx:',np.mean([cache[filename]['dqdx_fit_mean'] for filename in filenames if filename in cache]))
    print('sigma mean dqdx:',np.std([cache[filename]['dqdx_fit_mean'] for filename in filenames if filename in cache]))
    print('width dq/dx:',np.mean([cache[filename]['dqdx_fit_width'] for filename in filenames if filename in cache]))
    print('sigma width dq/dx:',np.std([cache[filename]['dqdx_fit_width'] for filename in filenames if filename in cache]))
    plt.figure('dqdx stability')
    ax = plt.subplot(2,1,1)
    x = list(map(dt.datetime.fromtimestamp,[(cache[filename]['run_start'] \
                                                 + cache[filename]['run_end'])/2 
                                            for filename in filenames if filename in cache]))
    xbar = list(map(lambda x: dt.timedelta(seconds=x),
                    [(cache[filename]['run_end'] - cache[filename]['run_start'])/2 
                        for filename in filenames if filename in cache]))
    timeloc = md.AutoDateLocator()
    timefmt = md.ConciseDateFormatter(timeloc)
    
    y = [cache[filename]['dqdx_fit_mean'] for filename in filenames if filename in cache]
    yerr = [np.sqrt(cache[filename]['dqdx_fit_cov'][1,1]) for filename in filenames if filename in cache]
    ax.errorbar(x,y,xerr=xbar,yerr=yerr, label='peak fit mean', fmt='.-', alpha=0.5)
    ax.set_ylabel('dQ/dx [ke/mm]')
    ax.set_xticks([])
    ax.axhline(expected_peak, color='r', label='expected')
    ax.legend()
    plt.grid(1)
    
    ax = plt.subplot(2,1,2,sharex=ax)
    y = [cache[filename]['dqdx_fit_width'] for filename in filenames if filename in cache]
    yerr = [np.sqrt(cache[filename]['dqdx_fit_cov'][2,2]) for filename in filenames if filename in cache]
    ax.errorbar(x,y,xerr=xbar,yerr=yerr, label='peak fit width', fmt='.-', alpha=0.5)
    ax.set_ylabel('dQ/dx [ke/mm]')
    ax.axhline(expected_sigma, color='r', label='expected')
    ax.legend()
    ax.xaxis.set_major_formatter(timefmt)
    plt.grid(1)
    plt.tight_layout()
    
    plt.figure('light efficiency')
    y = [cache[filename]['light_eff']*100 for filename in filenames if filename in cache]
    plt.errorbar(x,y,xerr=xbar, fmt='.-', alpha=0.5)
    plt.ylabel('light trigger efficiency [%]')
    ax = plt.gca()
    ax.xaxis.set_major_formatter(timefmt)
    plt.grid(1)
    plt.tight_layout()
    
    plt.figure('ext trigger rate')
    y = [cache[filename]['light_trigger_rate'] for filename in filenames if filename in cache]
    plt.errorbar(x,y,xerr=xbar, fmt='.-', alpha=0.5)
    plt.ylabel('ext trigger rate [Hz]')
    ax = plt.gca()
    ax.xaxis.set_major_formatter(timefmt)
    plt.grid(1)
    plt.tight_layout()
    
    plt.figure('muon rate')
    y = [cache[filename]['good_track_rate'] for filename in filenames if filename in cache]
    plt.errorbar(x,y,xerr=xbar, fmt='.-', alpha=0.5)
    plt.ylabel('muon rate [Hz]')
    ax = plt.gca()
    ax.xaxis.set_major_formatter(timefmt)
    plt.grid(1)
    plt.tight_layout()
    
    for filename in filenames:
        if filename not in cache: continue
        plt.figure(filename+' dqdx')
        vals,bins = cache[filename]['dqdx_dist']
        plt.hist(bins[:-1],bins=bins,weights=vals,histtype='step',label='data')
        x = np.linspace(*cache[filename]['dqdx_fit_range'],1000)
        y = gaus(x, cache[filename]['dqdx_fit_peak'], cache[filename]['dqdx_fit_mean'],
                 cache[filename]['dqdx_fit_width'])
        plt.plot(x,y,'-',label='fit')
        plt.title(filename)
        plt.ylabel('count')
        plt.xlabel('dQ/dx [ke/mm]')
        plt.grid(1)
        plt.legend()
        plt.tight_layout()
        
        plt.figure(filename+' track length')
        vals,bins = cache[filename]['track_length_dist']
        plt.hist(bins[:-1],bins=bins,weights=vals,histtype='step',label='data')
        plt.title(filename)
        plt.ylabel('count')
        plt.xlabel('track length [mm]')
        plt.grid(1)
        plt.legend()
        plt.tight_layout()
        
                

In [ ]:
## load M Mooney's lifetime file
import ROOT
f = ROOT.TFile('~/www_dune/data/Bern-singlecube/LArPix/electronLifetime/ElecLifetime_SingleCube_Bern.root','r')
lt = f.Get('lifetime')
x = list(map(dt.datetime.fromtimestamp,lt.GetX()))
y = list(lt.GetY())
f.Close()

In [ ]:
## plot lifetime over the run
import matplotlib.pyplot as plt
%matplotlib widget
plt.close('lifetime')
plt.figure('lifetime')
timeloc = md.AutoDateLocator()
timefmt = md.ConciseDateFormatter(timeloc)
plt.plot(x,y,'.-',alpha=0.5)
plt.ylabel('lifetime [ms]')
ax = plt.gca()
ax.xaxis.set_major_formatter(timefmt)
plt.grid(1)